## langchain入门

In [1]:
!pip install langchain

In [2]:
import os
api_key = os.getenv("OPENAI_API_KEY")

### langchain调用

In [3]:
#1首先导入LLM包装器，然后即可调用大语言模型
from langchain.llms import OpenAI

In [4]:
#初始化一个智能体名为llm
llm = OpenAI(temperature=0.9) #设置温度增加随机性
text = "What would be a good company name for a company that makes colorful socks?"
print(llm(text))

d:\Anaconda\envs\torchcpu\lib\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.llms.openai.OpenAI` was deprecated in langchain-community 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAI`.
  warn_deprecated(
d:\Anaconda\envs\torchcpu\lib\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.7 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(




"Rainbow Toes Socks" or "Vibrant Socks Co."


### 提示模版promptTemplate

In [6]:
#2导入提示模版的功能
from langchain.prompts import PromptTemplate

#写一个提示模版
prompt = PromptTemplate(
    input_variables=["product"],
    template="What is a good name for a company that makes {product}?",
)

#我们可以调用. format 方法来格式化它。
print(prompt.format(product="colorful socks"))

What is a good name for a company that makes colorful socks?


### 链：在多步骤的工作流中组合 LLM 和提示
最核心的链类型是 LLMChain，它由 PromptTemplate 和 LLM 组成。

扩展前面的示例，我们可以构造一个LLMChain.

它接受用户输入，使用 PromptTemplate 对其进行格式化，然后将格式化后的响应传递给LLM。

In [7]:
from langchain.prompts import PromptTemplate
from langchain.llms import OpenAI
#设定语言模型 
llm = OpenAI(temperature=0.9)
#设置提示模版，内含参数product
prompt = PromptTemplate(
    input_variables=["product"],
    template="What is a good name for a company that makes {product}?",
)

我们现在可以创建一个非常简单的链: 它接受用户输入，用它格式化提示符，然后将它发送到 LLM:

In [8]:
#3导入链的功能
from langchain.chains import LLMChain

chain = LLMChain(llm=llm, prompt=prompt)

#现在我们可以运行该链，只指定产品
chain.run("colorful socks")

d:\Anaconda\envs\torchcpu\lib\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The function `run` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


'\n\n"Rainbow Sole Co." or "ChromaSock Co." '

### 智能体 Agent: 基于用户输入的动态调用链
工具（tools): 执行特定任务的功能。这可以是: Google 搜索、数据库查找、 Python REPL、其他链。工具的接口目前是一个函数，预计将有一个字符串作为输入，一个字符串作为输出。

大语言模型（LLM）: 为代理提供动力的语言模型。

代理（agents）: 要使用的代理。这应该是引用支持代理类的字符串。因为本教程主要关注最简单、最高级别的 API，所以它只涉及使用标准支持的代理。如果要实现自定义代理，请参阅自定义代理的文档(即将发布)。

In [17]:
#安装用于Google搜索的工具
!pip install google-search-results

import os
os.getenv("OPENAI_API_KEY")
os.environ["SERPAPI_API_KEY"] = 'dfa474696c0dda7013044cbf30986da06aa12ffdc8bf2c655dc617c9b1852b19'

In [20]:
#4导入智能体中的调用工具和智能体功能
from langchain.agents import load_tools
from langchain.agents import initialize_agent
from langchain.agents import AgentType
#导入语言模型
from langchain.llms import OpenAI
 
#首先设置语言模型
llm = OpenAI(temperature=0)
 
#接下来加载一些工具，其中llm-math工具需要用到LLM，也得放进去
tools = load_tools(["serpapi", "llm-math"], llm=llm) #serpapi也需要api才能运行
#tools = load_tools(["llm-math"], llm=llm)
#最后初始化含有工具的智能体，以及我们想用要的智能体类型：零样本学习的反应式的描述型智能助手
#verbose：布尔值参数。如果为True，则代理在执行任务时将提供详细的反馈和说明。如果为False，代理可能只会返回在执行时产生的最终结果。
agent = initialize_agent(tools, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True)
 
# Now let's test it out!
agent.run("What was the high temperature in SF yesterday in Fahrenheit? What is that number raised to the .023 power?")



> Entering new AgentExecutor chain...
 I should use a search engine to find the high temperature in SF yesterday and then use a calculator to raise it to the .023 power.
Action: Search
Action Input: "SF high temperature yesterday"
Observation: ['See weather overview. San Francisco Temperature Yesterday. Maximum temperature yesterday: 58 °F (at 11:56 am) Minimum temperature yesterday: 53 °F (at 5:56 am)', 'Weather observations for the past three days, NWS logo. San Francisco, San ... Sky Cond. Temperature (ºF), Relative Humidity, Wind Chill (°F), Heat Index (°F) ...', 'High: 55.94ºf @2:56 AM Low: 55.04ºf @12:56 AM Approx. Precipitation / Rain Total: in. Time (PST), Temp. (ºf), Humidity (%), Dew Point ( ...', '37.78 °N, 122.42 °W. San Francisco, CA Weather Historystar_ratehome. icon 59 °F San Francisco International Airport Station|Report. Report Station.', 'San Francisco Bay Area Temperature Yesterday. Maximum temperature yesterday: 62 °F (at 12:16 pm) Minimum temperature yesterday: 5

'1.0978900365851625'

### 内存: 向链和代理添加状态
到目前为止，我们经历过的所有工具和代理都是无状态的的。

但是通常，您可能希望链或代理具有某种“内存”概念，以便它可以记住关于其以前的交互的信息。

最简单明了的例子就是在设计一个聊天机器人时——你想让它记住之前的消息，这样它就可以利用这些消息的上下文来进行更好的对话。

这是一种“短期记忆”。在更复杂的一面，你可以想象一个链条/代理随着时间的推移记住关键信息——这将是一种形式的“长期记忆”。

LangChain 提供了几个专门为此目的创建的链。 本教程使用其中一个链( ConversationChain ) 和两种不同类型的内存来完成操作。

默认情况下，, ConversationChain 有一个简单的内存类型，它记住所有以前的输入/输出，并将它们添加到传递的上下文中。

让我们看一下如何使用这个链(设置 verbose=True，这样我们就可以看到提示符)。

In [21]:
#5导入对话chain
from langchain import OpenAI, ConversationChain

llm = OpenAI(temperature=0)

#设置会记忆的链：ConversationChain
conversation = ConversationChain(llm=llm, verbose=True)

output = conversation.predict(input="Hi there!")
print(output)



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: Hi there!
AI:

> Finished chain.
 Hello! It's nice to meet you. I am an AI created by OpenAI. I am constantly learning and improving my abilities through machine learning algorithms. How can I assist you today?


In [22]:
output = conversation.predict(input="I'm doing well! Just having a conversation with an AI.")
print(output)



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Hi there!
AI:  Hello! It's nice to meet you. I am an AI created by OpenAI. I am constantly learning and improving my abilities through machine learning algorithms. How can I assist you today?
Human: I'm doing well! Just having a conversation with an AI.
AI:

> Finished chain.
 That's great to hear! I am always happy to engage in conversations and learn more about human interactions. Is there anything specific you would like to talk about?


### 构建语言模型应用程序: 聊天模型
### chat = ChatOpenAI(temperature=0)
聊天模型是语言模型的一种变体。

虽然聊天模型使用的是底层的语言模型，但它们公开的接口有些不同: 它们没有公开“文本输入、文本输出”API，而是公开了一个接口，其中“聊天消息”是输入和输出。

您可以通过向聊天模型传递一条或多条消息来完成聊天。

响应将是一条消息。

LangChain 中当前支持的消息类型是 AIMessage , HumanMessage , SystemMessage , 和 ChatMessage – ChatMessage 接受任意角色参数。大多数时候，您只需要处理 HumanMessage , AIMessage , 和 SystemMessage .

In [23]:
#6导入聊天chat模型
from langchain.chat_models import ChatOpenAI
from langchain.schema import (
    AIMessage,
    HumanMessage,
    SystemMessage
)
#（聊天）语言模型
chat = ChatOpenAI(temperature=0)

d:\Anaconda\envs\torchcpu\lib\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.chat_models.openai.ChatOpenAI` was deprecated in langchain-community 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(


您可以通过传入单个消息来完成

In [24]:
chat([HumanMessage(content="Translate this sentence from English to French. I love programming.")])

d:\Anaconda\envs\torchcpu\lib\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.7 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


AIMessage(content="J'adore la programmation.")

还可以为 OpenAI 的 gpt-3.5-turbo 和 gpt-4型号传递多条消息。

In [26]:
messages = [
    SystemMessage(content="You are a helpful assistant that translates English to Chinese."),
    HumanMessage(content="Translate this sentence from English to Chinese. I love programming.")
]
chat(messages)

AIMessage(content='我喜欢编程。')

您可以更进一步，使用generate为多组消息生成完成。

这将返回一个带有附加message参数的 LLMResult

In [31]:
batch_messages = [
    [
        SystemMessage(content="You are a helpful assistant that translates English to Chinese."),
        HumanMessage(content="Translate this sentence from English to Chinese. I love programming.")
    ],
    [
        SystemMessage(content="You are a helpful assistant that translates English to Chinese."),
        HumanMessage(content="Translate this sentence from English to Chinese. I love artificial intelligence.")
    ],
]
result = chat.generate(batch_messages)
 
print(result)
result.llm_output['token_usage']

generations=[[ChatGeneration(text='我喜欢编程。', generation_info={'finish_reason': 'stop', 'logprobs': None}, message=AIMessage(content='我喜欢编程。'))], [ChatGeneration(text='我喜欢人工智能。', generation_info={'finish_reason': 'stop', 'logprobs': None}, message=AIMessage(content='我喜欢人工智能。'))]] llm_output={'token_usage': {'completion_tokens': 19, 'prompt_tokens': 69, 'total_tokens': 88}, 'model_name': 'gpt-3.5-turbo'} run=[RunInfo(run_id=UUID('61a2d631-c96a-4661-8303-e6a30c506373')), RunInfo(run_id=UUID('58c17b29-2b7d-4b77-980b-a28fb47de8cf'))]


{'completion_tokens': 19, 'prompt_tokens': 69, 'total_tokens': 88}

### 聊天提示模版
与 LLM 类似，您可以通过使用 MessagePromptTemplate来使用模板。

可以从一个或多个 MessagePromptTemplate 生成 ChatPromptTemplate。

您可以使用 ChatPromptTemplate 的 format _ tip ——这将返回一个 PromptValue，

您可以将其转换为字符串或 Message 对象，具体取决于您是想将格式化的值用作 llm 或聊天模型的输入。

为了方便起见，在模板上公开了一个 from _ template 方法。如果你使用这个模板，它看起来是这样的:

In [33]:
#7导入聊天对应的输入模版
from langchain.chat_models import ChatOpenAI
from langchain.prompts.chat import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
)
#（聊天）语言模型
chat = ChatOpenAI(temperature=0)
#系统输入模版：对接两个参数
template = "You are a helpful assistant that translates {input_language} to {output_language}."
system_message_prompt = SystemMessagePromptTemplate.from_template(template)
#用户输入模版：对接text
human_template = "{text}"
human_message_prompt = HumanMessagePromptTemplate.from_template(human_template)
#chat负责合并系统和用户的
chat_prompt = ChatPromptTemplate.from_messages([system_message_prompt, human_message_prompt])
 
# get a chat completion from the formatted messages
chat(chat_prompt.format_prompt(input_language="English", output_language="Chinese", text="I love programming.").to_messages())

AIMessage(content='我喜欢编程。')

### 带聊天模型的链
上一节讨论的 LLMChain也可以用于聊天模型:

In [34]:
#8导入链+聊天模型结合
from langchain.chat_models import ChatOpenAI
from langchain import LLMChain
from langchain.prompts.chat import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
)
#（聊天）语言模型
chat = ChatOpenAI(temperature=0)
#构建输入模版
template = "You are a helpful assistant that translates {input_language} to {output_language}."
system_message_prompt = SystemMessagePromptTemplate.from_template(template)

human_template = "{text}"
human_message_prompt = HumanMessagePromptTemplate.from_template(human_template)

chat_prompt = ChatPromptTemplate.from_messages([system_message_prompt, human_message_prompt])

#构建chain
chain = LLMChain(llm=chat, prompt=chat_prompt)
chain.run(input_language="English", output_language="Chinese", text="I love programming.")
 
# -> "J'aime programmer."

'我喜欢编程。'

### 具有聊天模型的智能体
代理也可以与聊天模型一起使用，您可以使用 AgentType.CHAT_ZERO_SHOT_REACT_DESCRIPTION作为代理类型来初始化一个聊天模型。

In [45]:
#9导入智能体+聊天模型
from langchain.agents import load_tools
from langchain.agents import initialize_agent
from langchain.agents import AgentType
from langchain.chat_models import ChatOpenAI
from langchain.llms import OpenAI
 
#（聊天）语言模型
chat = ChatOpenAI(temperature=0)
 
#给智能体设置工具，其中llm-math需要用到一个llm，所以一起定义
llm = OpenAI(temperature=0)
tools = load_tools(["serpapi", "llm-math"], llm=llm)
 
#初始化智能体
agent = initialize_agent(tools, chat, agent=AgentType.CHAT_ZERO_SHOT_REACT_DESCRIPTION, verbose=True)
 
# Now let's test it out!
agent.run("Who is Messi? What is his current age raised to the 0.23 power?")



> Entering new AgentExecutor chain...
Question: Who is Messi? What is his current age raised to the 0.23 power?
Thought: I need to search for information about Messi and his current age.
Action:
```
{
  "action": "Search",
  "action_input": "Messi current age"
}
```

Observation: 36 years
Thought:I have found that Messi's current age is 36 years. Now I need to calculate his current age raised to the 0.23 power.
Action:
```
{
  "action": "Calculator",
  "action_input": "36^0.23"
}
```
Observation: Answer: 2.2800773226742175
Thought:

ValueError: An output parsing error occurred. In order to pass this error back to the agent and have it try again, pass `handle_parsing_errors=True` to the AgentExecutor. This is the error: Could not parse LLM output: I now know the final answer.

### 记忆内存: 向链和代理添加状态
您可以对链使用 Memory，对代理使用聊天模型进行初始化。

这与 LLM 的 Memory 之间的主要区别在于，我们不需要将以前的所有消息压缩成一个字符串，而是可以将它们保留为自己独特的内存对象。

In [48]:
#10导入记忆部分+聊天模型
from langchain.prompts import (
    ChatPromptTemplate, 
    MessagesPlaceholder, 
    SystemMessagePromptTemplate, 
    HumanMessagePromptTemplate
)
from langchain.chains import ConversationChain
from langchain.chat_models import ChatOpenAI
from langchain.memory import ConversationBufferMemory
#输入模版 
prompt = ChatPromptTemplate.from_messages([
    SystemMessagePromptTemplate.from_template("The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know."),
    MessagesPlaceholder(variable_name="history"),
    HumanMessagePromptTemplate.from_template("{input}")
])
#(聊天)语言模型
llm = ChatOpenAI(temperature=0)
#记忆设置
memory = ConversationBufferMemory(return_messages=True)

#拼接成链（记忆+输入+模型）
conversation = ConversationChain(memory=memory, prompt=prompt, llm=llm,verbose=True)

conversation.predict(input="Hi there!")
 
conversation.predict(input="I'm doing well! Just having a conversation with an AI.")
 
#conversation.predict(input="Tell me about yourself.")
 



> Entering new ConversationChain chain...
Prompt after formatting:
System: The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.
Human: Hi there!

> Finished chain.


> Entering new ConversationChain chain...
Prompt after formatting:
System: The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.
Human: Hi there!
AI: Hello! How can I assist you today?
Human: I'm doing well! Just having a conversation with an AI.

> Finished chain.


"That's great to hear! I'm here to chat and answer any questions you may have. What's on your mind?"